In [1]:
import numpy as np
import pandas as pd

from functools import partial

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
# # increase the number of characters shown for each column
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:

weather = pd.read_csv('./weather.csv', 
                      parse_dates=['Date']).drop('SnowFall', axis=1)


In [4]:
weather.head(2)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,0.00,29.18,29.82,2.7,25,9.6


In [5]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
       'ResultDir', 'AvgSpeed'],
      dtype='object')

In [6]:
### Depth, Water1 don't provide any valuable informaion.  The unique values for these columns are
#Depth ['0']
#Water1 ['M']
### moreover, we consider it highly doubtful that sunrise and sunset will have predictive value, so we are
### deleting these columns
to_drop = ['Depth', 'Water1','Sunrise', 'Sunset',]
weather.drop(to_drop,inplace=True,axis=1)




In [7]:
weather.head(2)


,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,,0.00,29.18,29.82,2.7,25,9.6


In [11]:
### what are the data types for each column?
for col in weather.columns:
    print (col, type(weather[col][0]))

Station <class 'numpy.int64'>
Date <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Tmax <class 'numpy.int64'>
Tmin <class 'numpy.int64'>
Tavg <class 'str'>
Depart <class 'str'>
DewPoint <class 'numpy.int64'>
WetBulb <class 'str'>
Heat <class 'str'>
Cool <class 'str'>
CodeSum <class 'str'>
PrecipTotal <class 'str'>
StnPressure <class 'str'>
SeaLevel <class 'str'>
ResultSpeed <class 'numpy.float64'>
ResultDir <class 'numpy.int64'>
AvgSpeed <class 'str'>


In [12]:
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                   object
Depart                 object
DewPoint                int64
WetBulb                object
Heat                   object
Cool                   object
CodeSum                object
PrecipTotal            object
StnPressure            object
SeaLevel               object
ResultSpeed           float64
ResultDir               int64
AvgSpeed               object
dtype: object

In [13]:
# for columns which contain strings, we want to know which can be converted into more useful data types and which 
# should remain as text .  Let's look at the contents of each column which contains strings:
for col in weather.columns:
    if type(weather[col][0]) == str:
        print (col, weather[col].unique())

Tavg ['67' '68' '51' '52' '56' '58' 'M' '60' '59' '65' '70' '69' '71' '61' '55'
 '57' '73' '72' '53' '62' '63' '74' '75' '78' '76' '77' '66' '80' '64'
 '81' '82' '79' '85' '84' '83' '50' '49' '46' '48' '45' '54' '47' '44'
 '40' '41' '38' '39' '42' '37' '43' '86' '87' '89' '92' '88' '91' '93'
 '94' '90' '36']
Depart ['14' 'M' '-3' ' 2' ' 4' ' 5' '10' '12' '13' '-2' '15' '11' '-4' '-6' ' 8'
 '-5' ' 1' ' 9' ' 6' '-9' '-8' ' 3' ' 0' '-1' '-7' ' 7' '-14' '18' '16'
 '22' '21' '20' '-10' '-16' '17' '-12' '-11' '-15' '-17' '19' '-13' '23']
WetBulb ['56' '57' '47' '48' '50' '49' '46' '54' '53' '62' '63' '60' '61' '51'
 '59' '58' '43' '44' '45' '65' '66' '67' '64' '52' '55' '69' '68' '70'
 '72' '71' '73' '74' '76' '42' '39' '40' '41' '35' '33' '34' 'M' '75' '36'
 '32' '38' '78' '77' '37']
Heat ['0' '14' '13' '9' '7' 'M' '5' '6' '4' '10' '8' '12' '3' '2' '1' '15' '16'
 '19' '17' '20' '11' '18' '21' '25' '24' '27' '26' '23' '28' '22' '29']
Cool [' 2' ' 3' ' 0' 'M' ' 5' ' 4' ' 6' ' 8' ' 7' ' 9' '10

In [ ]:
# CodeSum needs to be converted into dummy columns containing a 0 or 1 value
# for each classification code
# Non-numeric symbols need to be removed from WetBulb, StnPressure, SeaLevel, PrecipTotal

#AvgSpeed, Tavg, Depart, WetBulb, Heat, Cool, PrecipTotal, StnPressure, SeaLevel need to be converted to floats


In [14]:
def to_float(value):
    value = value.strip()
    value = value.replace('M','29.5')
    value = value.replace('T','0.0')
    value = float(value)
    return(value)

In [15]:

### converting string columns to integers
str_columns = ['AvgSpeed', 'Tavg', 'Depart', 'WetBulb', 'Heat', 'Cool', 'PrecipTotal', 'StnPressure', 'SeaLevel']
for col in str_columns:
    weather[col] = weather[col].apply(lambda x: to_float(x))

In [16]:
weather.head(10)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67.0,14.0,51,56.0,0.0,2.0,,0.0,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68.0,29.5,51,57.0,0.0,3.0,,0.0,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51.0,-3.0,42,47.0,14.0,0.0,BR,0.0,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52.0,29.5,42,47.0,13.0,0.0,BR HZ,0.0,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56.0,2.0,40,48.0,9.0,0.0,,0.0,29.39,30.12,11.7,7,11.9
5,2,2007-05-03,67,48,58.0,29.5,40,50.0,7.0,0.0,HZ,0.0,29.46,30.12,12.9,6,13.2
6,1,2007-05-04,66,49,58.0,4.0,41,50.0,7.0,0.0,RA,0.0,29.31,30.05,10.4,8,10.8
7,2,2007-05-04,78,51,29.5,29.5,42,50.0,29.5,29.5,,0.0,29.36,30.04,10.1,7,10.4
8,1,2007-05-05,66,53,60.0,5.0,38,49.0,5.0,0.0,,0.0,29.40,30.10,11.7,7,12.0
9,2,2007-05-05,66,54,60.0,29.5,39,50.0,5.0,0.0,,0.0,29.46,30.09,11.2,7,11.5


In [17]:
### creating new columns for dummies
codes = ['HZ', 'VC', 'FU', 'BC', 'SQ', 'FG+', 'MI', 'TS', 'DZ', 'RA', 'BR', 'FG', 'SN']
for code in codes:
    weather[code] = 0

In [18]:
def dummy_codes(x,code):
        if code in x:
            return 1
        else:
            return 0

In [20]:
for code in codes:
    weather[code] = weather['CodeSum'].apply(lambda x: dummy_codes(x,code))
weather.head(10)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,HZ,VC,FU,BC,SQ,FG+,MI,TS,DZ,RA,BR,FG,SN
0,1,2007-05-01,83,50,67.0,14.0,51,56.0,0.0,2.0,,0.0,29.10,29.82,1.7,27,9.2,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-01,84,52,68.0,29.5,51,57.0,0.0,3.0,,0.0,29.18,29.82,2.7,25,9.6,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-02,59,42,51.0,-3.0,42,47.0,14.0,0.0,BR,0.0,29.38,30.09,13.0,4,13.4,0,0,0,0,0,0,0,0,0,0,1,0,0
3,2,2007-05-02,60,43,52.0,29.5,42,47.0,13.0,0.0,BR HZ,0.0,29.44,30.08,13.3,2,13.4,1,0,0,0,0,0,0,0,0,0,1,0,0
4,1,2007-05-03,66,46,56.0,2.0,40,48.0,9.0,0.0,,0.0,29.39,30.12,11.7,7,11.9,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2,2007-05-03,67,48,58.0,29.5,40,50.0,7.0,0.0,HZ,0.0,29.46,30.12,12.9,6,13.2,1,0,0,0,0,0,0,0,0,0,0,0,0
6,1,2007-05-04,66,49,58.0,4.0,41,50.0,7.0,0.0,RA,0.0,29.31,30.05,10.4,8,10.8,0,0,0,0,0,0,0,0,0,1,0,0,0
7,2,2007-05-04,78,51,29.5,29.5,42,50.0,29.5,29.5,,0.0,29.36,30.04,10.1,7,10.4,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,2007-05-05,66,53,60.0,5.0,38,49.0,5.0,0.0,,0.0,29.40,30.10,11.7,7,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2,2007-05-05,66,54,60.0,29.5,39,50.0,5.0,0.0,,0.0,29.46,30.09,11.2,7,11.5,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:

#dropping original CodeSum column
weather = weather.drop('CodeSum', axis=1)

In [22]:
### columns where we want an average
columns_to_avg = ['Tavg','DewPoint','WetBulb','PrecipTotal','StnPressure','SeaLevel','AvgSpeed','HZ', 'VC', 'FU', 'BC',
       'SQ', 'FG+', 'MI', 'TS', 'DZ', 'RA', 'BR', 'FG', 'SN']
columns_to_max = ['Tmax','Heat','ResultSpeed']
columns_to_min = ['Tmin','Cool']
columns_to_sum = ['HZ', 'VC', 'FU', 'BC',
       'SQ', 'FG+', 'MI', 'TS', 'DZ', 'RA', 'BR', 'FG', 'SN']

for i in columns_to_avg:
    weather[i] = weather[i].rolling(window=30, center=False,min_periods = 1).mean()
for i in columns_to_max:
    weather[i] = weather[i].rolling(window=30, center=False,min_periods = 1).max()
for i in columns_to_min:
    weather[i] = weather[i].rolling(window=30, center=False,min_periods = 1).min()

In [23]:
weather.tail(10)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,HZ,VC,FU,BC,SQ,FG+,MI,TS,DZ,RA,BR,FG,SN
2934,1,2014-10-27,77.0,33.0,54.666667,16.0,45.066667,49.933333,22.0,0.0,0.103000,29.223667,29.923667,12.0,19,8.920000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300000,0.300000,0.433333,0.0,0.000000
2935,2,2014-10-27,79.0,33.0,55.066667,29.5,45.333333,50.233333,22.0,0.0,0.103333,29.213333,29.912667,12.7,19,9.043333,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.266667,0.300000,0.400000,0.0,0.000000
2936,1,2014-10-28,79.0,33.0,54.900000,10.0,44.666667,49.800000,22.0,0.0,0.082000,29.219333,29.916667,14.8,26,9.260000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.233333,0.266667,0.366667,0.0,0.000000
2937,2,2014-10-28,79.0,33.0,54.633333,29.5,43.966667,49.333333,22.0,0.0,0.068333,29.226000,29.920667,14.8,26,9.450000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.233333,0.266667,0.333333,0.0,0.000000
2938,1,2014-10-29,79.0,33.0,54.100000,-4.0,43.133333,48.733333,22.0,0.0,0.046667,29.243667,29.937667,14.8,29,9.440000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.233333,0.300000,0.0,0.000000
2939,2,2014-10-29,79.0,33.0,53.566667,29.5,42.300000,48.133333,22.0,0.0,0.007333,29.261333,29.955333,14.8,29,9.453333,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.200000,0.266667,0.0,0.000000
2940,1,2014-10-30,79.0,32.0,53.200000,-4.0,41.733333,47.766667,23.0,0.0,0.005667,29.271000,29.966667,14.8,24,9.306667,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.166667,0.233333,0.0,0.000000
2941,2,2014-10-30,79.0,32.0,52.833333,29.5,41.133333,47.366667,23.0,0.0,0.003333,29.281333,29.978667,14.8,23,9.200000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.166667,0.200000,0.0,0.000000
2942,1,2014-10-31,79.0,32.0,52.400000,-6.0,40.333333,46.766667,25.0,0.0,0.004333,29.297000,29.993000,22.6,34,9.716667,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.200000,0.166667,0.0,0.033333
2943,2,2014-10-31,79.0,32.0,51.933333,29.5,39.566667,46.200000,25.0,0.0,0.005667,29.312000,30.007333,22.6,34,10.190000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.233333,0.166667,0.0,0.066667


In [24]:
weather['FG+'].value_counts()

0.000000    2506
0.033333     191
0.066667     166
0.133333      44
0.100000      37
Name: FG+, dtype: int64

In [25]:
weather.dropna().shape

(2944, 29)

In [26]:
weather.shape


(2944, 29)